In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [3]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  regex = re.compile('[^A-Za-zА-Яа-я]')
  cleantext = regex.sub(' ', cleantext)
  return cleantext

In [5]:
def gettext(bad): 
    mid = []
    i = 1
    while bad[i].text.split()[0] != 'Материал' and bad[i].text.split()[1] != 'опубликован':
        mid.append(bad[i].text)
        i += 1
        if i > len(mid):
            break
    good = [j for i in mid for j in i.split()]
    return good

In [ ]:
def getli(li):
    mid = []
    

In [6]:
def document(id):
    page = requests.get('https://vc.ru/' + str(id) )
    soup = BeautifulSoup(page.text, 'html5lib')
    info = []
    if soup.find("h1", class_="content-header__title") and 'Вакансия' not in soup.find('title').text:
        #titledone
        info.append(soup.find("h1", class_="content-header__title").text.split())
        #text
        if soup.find_all('p') != None:
            bad = soup.find_all('p')
            info.append(gettext(bad))
        if soup.find_all("li") != None: 
            info.append([var for i in soup.find_all("li") for var in i.text.split()])
        else:
            info.append('NaN')
        #yeardone
        info.append(soup.find("time", class_= "time").text.split()[2])
        #sectiondone
        info.append(soup.find("div", class_=
                              "content-header-author__name").text.split())
        #votesdone
        a = soup.find("span", class_="vote__value__v vote__value__v--real").text
        if a[0] == '–':
            b = int(a[1:])
            info.append(-b)
        else:
            info.append(int(a))
        #viewsdone
        info.append(soup.find("span", class_="views__value").text)
        #bookmarksdone
        info.append(int(soup.find('div',
                        class_="favorite_marker favorite_marker--type-content favorite_marker--non_zero").text.split()[0]))
        #commentsdone
        comment1 = soup.find_all("div", class_="comments__item__text")
        comments = []
        for i in range(len(comment1)):
            comments.append(comment1[i].text.split())
        info.append(comments)
    else:
        info.append('NaN')
    print(id)
    return info

In [7]:
a = list(map(document, np.arange(55000,55023)))
df = pd.DataFrame(a, columns=['title', 'text', 'year', 'section', 'votes', 'views', 'bookmarks', 'comments'])
df.dropna(inplace=True)
df

55000
55001
55002
55003
55004
55005
55006


KeyboardInterrupt: 